# Converting 2P data with ophys and pupillometry to NWB format

- Methods here specific to PC data in Cody et al 2024 (doi: 10.1523/JNEUROSCI.0939-23.2024), but can be abstracted to other experiments.
- Requires subject metadata in `./data/animalList.csv` and experiment metadata in `./data/experimentMetadata.csv`
- Explore NWB output file with `neurosift`
    1. in python env: `pip install --upgrade neurosift`
    2. `neurosift view-nwb AA0304_DANDI.nwb`
- After conversion, confirm NWB format conforms to DANDI standards w/ nwbinspector:
    1. in python env: `pip install -U nwbinspector`
    2. `nwbinspector "AA0304.nwb" --config dandi`

- For pupillometry tables in .mat files, require matlab component to reformat table as struct to be able to load into python. See: `./extra/tableMAT2StructMAT.m`.

## Run NWB data standardization function on a single experiment:

In [ ]:
import os
import lib.nwbScanImage

experimentID = 'AA0314'
dataPath = "/media/DATA/backups/sutter2P_backup/D_drive/dataSince20210304"
outputNWBpath = os.path.join(dataPath,experimentID,f"{experimentID}_fix.nwb")

session_description = f"pupilPTinContrast_{experimentID}_C57BL6J"
experiment_description = "Pure tone in low and high DRC contrast with pupillometry"
keywords = ["2P", "CGC", "pupillometry", "DRC"]

subject = lib.nwbScanImage.setSubject(
    subject_id=experimentID,
    age="P47D",
    species="Mus musculus",
    sex="F",
    genotype="C57BL6/J",
    description="Injected on 4/7/2021 with AAV9.CaMKII.GCaMP6f.WPRE.SV40 at 1:6 into right ACtx"
)

lib.nwbScanImage.genNWBfromScanImage_pc(
    experimentID=experimentID, 
    dataPath=dataPath, 
    NWBoutputPath=outputNWBpath,
    subject=subject,
    session_description=session_description,
    experiment_description=experiment_description,
    keywords=keywords,
    **lib.nwbScanImage.PARAMS_nwbFilePC,
    **lib.nwbScanImage.PARAMS_imagingPC
)

## Run NWB build over multiple experiments/directories:
- Example shown below, but can run on commandline.
- Data path is assumed to have folders organized by experimentID.
1. Activate python venv: `source ./venvNWB/bin/activate`
2. Call python script with dataPath as argument: `python ./scanimage2nwb.py [dataPath]`

In [17]:
# retrieve subject and experiment metadata from log tables
import pandas as pd
import ast

subjects = pd.read_csv('./data/animalList.csv')
subjects = subjects.set_index('subject_id')
experiments = pd.read_csv('./data/experimentMetadata.csv')
experiments = experiments.set_index('experimentID')
experiments['keywords'] = experiments['keywords'].apply(ast.literal_eval)


In [18]:
subjects.head()

,DOB,age,genotype,sex,Virus,Inj. Date,dilution,description
subject_id,,,,,,,,
AA0304,1/21/21,49,C57BL6/J,F,AAV9.CaMKII. GCaMP6f.WPRE.SV40,2/22/21,1:6,Injected on 2/22/21 with AAV9.CaMKII. GCaMP6f....
AA0310,3/4/21,45,C57BL6/J,M,AAV9.CaMKII. GCaMP6f.WPRE.SV40,4/1/21,1:6,Injected on 4/1/21 with AAV9.CaMKII. GCaMP6f.W...
AA0308,3/4/21,41,C57BL6/J,F,AAV9.CaMKII. GCaMP6f.WPRE.SV40,4/1/21,1:6,Injected on 4/1/21 with AAV9.CaMKII. GCaMP6f.W...
AA0314,3/9/21,47,C57BL6/J,F,AAV9.CaMKII. GCaMP6f.WPRE.SV40,4/7/21,1:6,Injected on 4/7/21 with AAV9.CaMKII. GCaMP6f.W...
AA0312,3/9/21,44,C57BL6/J,M,AAV9.CaMKII. GCaMP6f.WPRE.SV40,4/7/21,1:6,Injected on 4/7/21 with AAV9.CaMKII. GCaMP6f.W...


In [19]:
experiments.head()

,session_description,experiment_description,keywords
experimentID,,,
AA0304,pupilPTinContrast,Pure tone in low and high DRC contrast with pu...,"[pupillometry, 2P, CGC, DRC]"
AA0308,pupilPTinContrast,Pure tone in low and high DRC contrast with pu...,"[pupillometry, 2P, CGC, DRC]"
AA0310,pupilPTinContrast,Pure tone in low and high DRC contrast with pu...,"[pupillometry, 2P, CGC, DRC]"
AA0312,pupilPTinContrast,Pure tone in low and high DRC contrast with pu...,"[pupillometry, 2P, CGC, DRC]"
AA0314,ZX1_uBasePupilPctMax_wEpoch | pupilPTinContras...,Pupil baseline re recording duration in ZX1 | ...,"[DRC, 2P, CGC, ZX1, pupillometry]"


In [ ]:
import os
import lib.nwbScanImage

# set datapath
dataPath = "/media/DATA/backups/sutter2P_backup/D_drive/Cody_et_al_2024"

for experimentID,d in experiments.iterrows():
    print(experimentID)

    outputNWBpath = os.path.join(dataPath,experimentID,f"{experimentID}_DANDI.nwb")
    
    subject = lib.nwbScanImage.setSubject(
        subject_id=experimentID,
        age=f"P{subjects.loc[experimentID]['age']}D",
        species="Mus musculus",
        sex=subjects.loc[experimentID]['sex'],
        genotype=subjects.loc[experimentID]['genotype'],
        description=subjects.loc[experimentID]['description']
    )

    lib.nwbScanImage.genNWBfromScanImage_pc(
        experimentID=experimentID, 
        dataPath=dataPath, 
        NWBoutputPath=outputNWBpath,
        subject=subject,
        session_description=d['session_description'],
        experiment_description=d['experiment_description'],
        keywords=d['keywords'],
        **lib.nwbScanImage.PARAMS_nwbFilePC,
        **lib.nwbScanImage.PARAMS_imagingPC
        )